<a href="https://colab.research.google.com/github/jyanivaddi/ERA_V1/blob/master/session_10/s10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download our code repository and install python dependencies

In [ ]:
#!git clone "https://github.com/jyanivaddi/ERA_V1.git"
#!git pull
#!pip install torchinfo
#!pip install prettytable
#!pip install torch_lr_finder

Add all the imports:

In [1]:
from __future__ import print_function
import sys
#sys.path.append("ERA_V1/session_10")
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
import albumentations as A
import albumentations.augmentations as AA
from albumentations.pytorch import ToTensorV2
from unet_model import UNet
from data_loader import load_oxford_pet_data

Allocate GPU and print model summary

In [5]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
unet_model = UNet(in_channels=3,out_channels=2).to(device)
#model_summary(unet_model, input_size=(3,48,48))


Define Albumentations image augmentations. We use RandomResizedCrop, HorizontalFlip, and cutout

Define batch size, train and test loaders

In [6]:
torch.manual_seed(1)
batch_size = 32
kwargs = {'num_workers': 2, 'pin_memory': True} if use_cuda else {}
train_loader, test_loader, class_names = load_oxford_pet_data(train_transforms, test_transforms, batch_size, **kwargs)

NameError: name 'train_transforms' is not defined

Preview Augmentations

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets, transforms

def un_normalize_image(img):
    un_norm_transform = transforms.Compose([transforms.Normalize((0.,0.,0.,),(1./0.247,1./0.244,1./0.262)),
                                                 transforms.Normalize((-0.491,-0.482,-0.447),(1.0,1.0,1.0))])
    return un_norm_transform(img)

def preview_augmentations(train_loader, image_transform):
    batch_data, batch_label = next(iter(train_loader))
    preview_img = np.asarray(un_normalize_image(batch_data[0].squeeze()))
    preview_img = (preview_img*255./np.max(preview_img)).astype('uint8')
    preview_label = batch_label[0]
    fig, axs = plt.subplots(1, 2)
    plt.tight_layout()

    un_normalized_img = un_normalize_image(batch_data[0].squeeze())
    un_normalized_img = np.asarray(un_normalized_img)
    transformed_numpy_img = image_transform(image = un_normalized_img.transpose(1,2,0))["image"]
    axs[0].imshow(preview_img.transpose((1,2,0)))
    axs[0].set_xticks([])
    axs[0].set_yticks([])
    axs[1].imshow(transformed_numpy_img)
    axs[1].set_xticks([])
    axs[1].set_yticks([])
    plt.tight_layout()
    plt.show()

# Reload train and test loader to preview augmentations
torch.manual_seed(1)
batch_size = 512
kwargs = {'num_workers': 2, 'pin_memory': True} if use_cuda else {}
eg_train_loader, eg_test_loader, eg_class_names = load_cifar10_data(train_transforms= A.Compose([A.Normalize(mean=(0.491,0.482,0.447),std=(0.247,0.244,0.262)),ToTensorV2()]), test_transforms=A.Compose([A.Normalize(mean=(0.491,0.482,0.447),std=(0.247,0.244,0.262)),ToTensorV2()]), batch_size=32, **kwargs)

# Random Resized Crop
img_transforms = A.Compose([AA.crops.transforms.RandomResizedCrop(height= 32,width = 32,p=0.2)])
preview_augmentations(eg_train_loader, img_transforms)

# Horizontal Flip
img_transforms = A.Compose([A.HorizontalFlip(always_apply=True)])
preview_augmentations(eg_train_loader, img_transforms)

# Cut out
img_transforms = A.Compose([AA.dropout.coarse_dropout.CoarseDropout(max_holes = 1, max_height=8,
                                                max_width=8, min_holes = 1,
                                                min_height=8, min_width=8,
                                                fill_value=(0.491, 0.482, 0.447),
                                                mask_fill_value = None, always_apply=True)])
preview_augmentations(eg_train_loader, img_transforms)



Show sample Images from the dataset

In [ ]:
preview_images(train_loader,class_names, num_rows = 5, num_cols = 5)

Train model using Batch Normalization

In [ ]:
drop_out = 0.02
num_epochs=24
resnet_model = CustomResnet(base_channels=3, num_classes=10,drop_out_probability=drop_out).to(device)
criterion = nn.CrossEntropyLoss(reduction='mean')
optim_obj = Optimization(resnet_model, device, train_loader, criterion, num_epochs)
lr_history = find_best_lr(resnet_model, train_loader, optim_obj.optimizer, criterion, device)


In [ ]:
train_losses = []
test_losses = []
train_acc = []
test_acc = []
lr_values = []
max_lr = 4.65e-2
# Define scheduler
optim_obj.define_scheduler(max_lr)

for epoch in range(1,num_epochs+1):
    lr_values.append(optim_obj.scheduler.get_lr())
    print(f"epoch: {epoch}\t learning rate: {optim_obj.scheduler.get_last_lr()[0]}")
    this_train_loss = training_loop(resnet_model, device, train_loader, optim_obj.optimizer, optim_obj.scheduler, criterion, train_acc, train_losses)
    this_loss = model_test(resnet_model, device, test_loader, criterion, test_acc, test_losses)
    #optim_obj.scheduler.step()



Plot LR values

In [ ]:
import matplotlib.pyplot as plt
def plot_lr_values2(lr_list):
    num_epochs = len(lr_list)
    plt.figure()
    plt.plot(range(1,num_epochs+1),lr_list)
    plt.xlabel('Epoch #')
    plt.ylabel("Learning Rate")
    #plt.yscale('log')
    plt.show()

def plot_lr_values(scheduler, num_epochs, num_batches):
    lrs = []
    steps = []
    for epoch in range(num_epochs):
        for batch in range(num_batches):
            scheduler.step()
            lrs.append(scheduler.get_last_lr()[0])
            steps.append(epoch * num_batches + batch)

    plt.figure()
    plt.legend()
    plt.plot(steps, lrs, label='OneCycle')
    plt.show()

def plot_losses(train_losses, test_losses):
    fig, axs = plt.subplots(1, 2, figsize=(15, 10))
    epochs = range(1,len(train_losses)+1)
    axs[0].plot(epochs, train_losses)
    axs[0].set_title("Train")
    axs[1].plot(epochs, test_losses)
    axs[1].set_title("Test")

def plot_accuracy(train_acc, test_acc, target_test_acc = 90.):
    epochs = range(1,len(train_acc)+1)
    plt.figure()
    plt.plot(epochs, train_acc, epochs, test_acc)
    plt.axhline(target_test_acc, color='r')
    plt.legend(('Train','Test'),loc='best')
    plt.title("Accuracy")


Plot results for Batch Norm experiment

In [ ]:
this_scheduler = OneCycleLR(
        optim_obj.optimizer,
        max_lr = max_lr,
        steps_per_epoch=len(optim_obj.train_loader),
        epochs = num_epochs,
        pct_start = 5./num_epochs,
        div_factor=200,
        three_phase=False,
        #final_div_factor=1000,
        anneal_strategy='linear',
        verbose=False
        )
plot_lr_values(this_scheduler, num_epochs, len(train_loader))
plot_losses(train_losses, test_losses)
plot_accuracy(train_acc, test_acc, target_test_acc=90.)


Print Training Log

In [ ]:
print_train_log(train_acc, test_acc, train_losses, test_losses)